<a href="https://www.kaggle.com/code/dascient/daily-crypto-buy-sell-decision-maker?scriptVersionId=123317819" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Updates daily - 08:00 CST. 
## An automated solution for lazy Binance.US traders that depend solely on technical analysis. 

### You're welcome :)

**Scheduled notebook**

This notebook is scheduled to run everyday at 08:00 CST to collect 300-day historical price actions on all Binance.US tradeable assets. It employs a plethoric set of technical indicators such as a kalman filter modeler, ichimoku forecasting condtionals, as well as, other complex algorithms in order to analyze and identify profitable entry & exit points.

### *POTENTIAL UPCOMING OUTAGE*
Due to Binance US having made some important API changes. Certain consequences may impact this notebook's ability to query historical transaction data for USD trading pairs, including trades and orders, that were executed before our system upgrade on February 5, 2023 at 9 p.m. PST / midnight EST.

To query transaction information for historical trades and orders, involving USD trading pairs, executed before the planned system upgrade, we will need to begin using the USD4 symbol. For example: "BTCUSD4".
To query transaction information for trades and orders, involving USD trading pairs, executed after the planned system upgrade, we would need to continue using the USD symbol. For example: “BTCUSD”.

***PLEASE STANDBY FOR MINOR UPDATES***

In [1]:
# login to binance
from IPython.display import clear_output

!pip install ccxt
!pip install schedule
!pip install pykalman
!pip install pandas_ta
!pip install plotly

import ccxt,schedule,warnings,time,ast
from pykalman import KalmanFilter
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from rich import print, pretty
from datetime import datetime
from random import randint
from random import seed
import pandas_ta as ta
import pandas as pd
import numpy as np
pretty.install()
import os

clear_output()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# this key pair only works for me, but if you'd like to run on your own binance api you can obtain yours by following:
# https://youtu.be/Q4Lt1KulkOQ

ccxt.binanceus({ 'options':{ 'adjustForTimeDifference':True}})
exchange = ccxt.binanceus({
"apiKey": "n0Oiypu18lqOHAkROTpYRUBNaoFWzIomdNk36S8g5xMPOUwhp8ZozRg4eMQ32Cl2",
"secret": "BzXJTwRBZuRNUViZZWoAvTdykXBl2wvsQ6hyUWI4deEJiQdR48u0WNqOBFY4GdT8",
'enableRateLimit': True})

# Calculate Indicators

In [3]:
# indicators + trade signals
def calculate_indicator(symbol):

    bars = exchange.fetch_ohlcv(symbol, timeframe='1d', limit=300)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize(None)
    
    close = df['close'][len(df)-1]
    low = df['low'][len(df)-1]
        
        
    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                      observation_matrices = [1],   # The value for Ht.
                      initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                      initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                      observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                      transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0
    # Get the Kalman smoothing
    state_means, _ = kf.filter(df['close'].values)
    # Call it kf_mean
    df['kf_mean'] = np.array(state_means)
    kalman = df.kf_mean[len(df)-1]
    aboveKalman = low > kalman
    
    
    # exponential moving averages 
    ema_14 = df.ta.ema(14, append=True)[-1:].reset_index(drop=True)[0]
    ema_91 = df.ta.ema(91, append=True)[-1:].reset_index(drop=True)[0]
    ema_125 = df.ta.ema(125, append=True)[-1:].reset_index(drop=True)[0]
    ema_crossover = ema_14 > ema_91
    
    
    # lower/upper 14-day bollinger bands for mean reversion
    bbl_14 = df.ta.bbands(length=14, append=True)[['BBL_14_2.0']].tail(1).values[0][0]
    bbu_14 = df.ta.bbands(length=14, append=True)[['BBU_14_2.0']].tail(1).values[0][0]
    bband_buy = close < bbl_14
    bband_sell = close > bbu_14
    
    
    # ichimoku 9 & 26-day forecasts 
    # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.trend.IchimokuIndicator
    isa_9 = df.ta.ichimoku()[1]['ISA_9'].tail(1).values[0] # help(ta.ichimoku)
    isb_26 = df.ta.ichimoku()[1]['ISB_26'].tail(1).values[0]

    
    # archer ma 
    # https://github.com/twopirllc/pandas-ta#general
    amat = (df.ta.amat()['AMATe_LR_8_21_2'].tail(1).values[0] == 1)

    
    # rsi
    rsi = df.ta.rsi()[len(df)-1]
    rsi_buy = rsi < 30
    rsi_sell = rsi > 70

    
    # choppy
    # https://github.com/twopirllc/pandas-ta#trend-18
    try: 
        chop = "{:.2f}".format(df.ta.chop()[len(df.ta.chop())-1]) 
    except RunTimeWarning:
        chop = 0


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # signal
    #buy = (close < isa_9) & (close < isb_26) & amat & rsi_buy & bband_buy & aboveKalman
    buy = amat & ema_crossover & aboveKalman
    
    #sell = (close > isa_9) & (close > isb_26) & ~amat & rsi_sell & bband_sell & ~aboveKalman
    sell = ~amat & ~ema_crossover & ~aboveKalman

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

    return df, symbol, close, isa_9, isb_26, chop, rsi, amat, ema_crossover, buy, sell, aboveKalman


# plotter
def plot(symbol):

    df = calculate_indicator(symbol)[0]

    fig = go.Figure(go.Candlestick(x=df.index,
                                   open=df['open'],
                                   high=df['high'],
                                   low=df['low'],
                                   close=df['close'],
                                   name=f'{symbol}'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['kf_mean'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='Kalman Filter'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_91'], 
                             opacity=0.7, 
                             line=dict(color='purple', width=2), 
                             name='EMA-91'))
    
    #fig.update_layout(title=f'{symbol}')
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig.show()


# color in dataframe
def color_boolean(val):
    color =''
    if val == True:
        color = 'lightgreen'
    elif val == False:
        color = 'pink'
    elif type(val) == str or float or int:
        color = 'lightblue'
    return 'background-color: %s' % color

# Prepare Data

In [4]:
today = pd.Timestamp(datetime.now()).strftime("%Y-%m-%d")
print(f"Calculating indicators for today's date: {today}.")

Calculating indicators for today's date: 2023-03-25.

In [5]:
# load symbols + initiate dataframe
results = []
symbols = exchange.fetchTickers().keys()

# iteration
for symbol in symbols:
    try:
        output = calculate_indicator(symbol)
        results.append({'Symbol':output[1],
                        'Buy Signal':output[9],
                        'Sell Signal':output[10],
                        'Date':today,
                        'Close':output[2],
                        'Ichimoku 9-Day Forecast':output[3],
                        'Ichimoku 26-Day Forecast':output[4],
                        'Choppiness (%)':output[5],
                        'RSI':output[6],
                        'Archer MA Trending':output[7],
                        'EMA14 > EMA91':output[8],
                        'Low > Kalman':output[11],
                        })
    except:
        pass
    
# frame & clean
results = pd.DataFrame(results)
results['Tick'] = [v.split('/')[0] for i,v in results.Symbol.items()]
results['Deno'] = [v.split('/')[1] for i,v in results.Symbol.items()]

# set pandas display to respect number of symbols in results
size = len(results)
pd.set_option('display.max_rows', size)

In [6]:
# get only 10 top buy & 10 top sell
df_top = results.sort_values(['Buy Signal','EMA14 > EMA91','Archer MA Trending'],ascending=False)
top10 = df_top[df_top["Tick"].str.contains("USD")==False].sort_values(['RSI'],ascending=True).drop_duplicates('Tick').head(10).reset_index(drop=True)
top20 = df_top[df_top["Tick"].str.contains("USD")==False].sort_values(['RSI'],ascending=True).drop_duplicates('Tick').head(20).reset_index(drop=True)

df_bottom = results.sort_values(['Sell Signal','Archer MA Trending','RSI'],ascending=False)
bottom10 = df_bottom[df_bottom["Tick"].str.contains("USD")==False].drop_duplicates('Tick').head(10).reset_index(drop=True)
bottom20 = df_bottom[df_bottom["Tick"].str.contains("USD")==False].drop_duplicates('Tick').head(20).reset_index(drop=True)

# Buying Opportunities

In [7]:
if any(top20['Buy Signal'] == True): 
    print("The charts below are assets that were identified as reasonable 'entry points' - sorted by ema crossovers, trends, & rsi values.")
    print("Great entry points for:")
    [print(y) for y in top10[top10['Buy Signal'] == True].Symbol.items()]
    
if all(top20['Buy Signal'] == False): 
    print("No recommended long positions today. Showing results sorted by significance of crossovers, trends, & rsi values.")
    

No recommended long positions today. Showing results sorted by significance of crossovers, 
trends, & rsi values.

In [8]:
top20.style.applymap(color_boolean)

,Symbol,Buy Signal,Sell Signal,Date,Close,Ichimoku 9-Day Forecast,Ichimoku 26-Day Forecast,Choppiness (%),RSI,Archer MA Trending,EMA14 > EMA91,Low > Kalman,Tick,Deno
0,BCH/BTC,False,True,2023-03-25,0.004430,0.004861,0.005181,49.48,21.432144,False,False,False,BCH,BTC
1,HNT/USD,False,True,2023-03-25,1.310000,1.964600,2.310000,41.80,24.254334,False,False,False,HNT,USD
2,UNI/BTC,False,True,2023-03-25,0.000215,0.000240,0.000261,37.21,26.733256,False,False,False,UNI,BTC
3,ATOM/BTC,False,True,2023-03-25,0.000410,0.000471,0.000554,44.24,26.745471,False,False,False,ATOM,BTC
4,DOT/BTC,False,True,2023-03-25,0.000220,0.000241,0.000263,42.89,28.051321,False,False,False,DOT,BTC
5,MATIC/BTC,False,True,2023-03-25,0.000040,0.000028,0.000118,30.26,28.308794,False,False,False,MATIC,BTC
6,TRX/BTC,False,True,2023-03-25,0.000002,0.000003,0.000003,42.44,29.017743,False,False,False,TRX,BTC
7,SOL/BTC,False,True,2023-03-25,0.000751,0.000827,0.000919,48.02,30.316354,False,False,False,SOL,BTC
8,VET/BTC,False,True,2023-03-25,0.000001,0.000001,0.000001,52.16,30.472234,False,False,False,VET,BTC
9,MANA/BTC,False,True,2023-03-25,0.000021,0.000024,0.000028,51.99,31.772370,False,False,False,MANA,BTC


In [9]:
# plots for top10
[plot(symbol) for i,symbol in top10.Symbol.items()]

[None, None, None, None, None, None, None, None, None, None]

# Selling Opportunities

In [10]:
if any(bottom20['Sell Signal'] == True): 
    print("The charts below are assets that were identified as reasonable 'exit points' - sorted by crossovers, trends, & rsi values.")
    print("Best selling points are at:")
    [print(y) for y in bottom10[bottom10['Sell Signal'] == True].Symbol.items()]
        
    
if all(bottom20['Sell Signal'] == False): 
    print("No recommended short positions today. Showing results sorted by significance of crossovers, trends, & rsi values.")
    

The charts below are assets that were identified as reasonable 'exit points' - sorted by 
crossovers, trends, & rsi values.

Best selling points are at:

(0, 'HNT/USD4')

(1, 'SANTOS/USD4')

(2, 'EGLD/USD')

...

(5, 'LAZIO/USD4')

(6, '1INCH/USD')

(7, 'MANA/USD')

(8, 'GAL/USD')

(9, 'JASMY/USDT')

In [11]:
bottom20.style.applymap(color_boolean)

,Symbol,Buy Signal,Sell Signal,Date,Close,Ichimoku 9-Day Forecast,Ichimoku 26-Day Forecast,Choppiness (%),RSI,Archer MA Trending,EMA14 > EMA91,Low > Kalman,Tick,Deno
0,HNT/USD4,False,True,2023-03-25,2.796400,2.749900,2.472050,70.78,52.918649,False,False,False,HNT,USD4
1,SANTOS/USD4,False,True,2023-03-25,5.685000,5.744000,5.151000,57.02,52.384632,False,False,False,SANTOS,USD4
2,EGLD/USD,False,True,2023-03-25,42.600000,44.258750,45.106000,53.57,47.813104,False,False,False,EGLD,USD
3,VITE/USDT,False,True,2023-03-25,0.023230,0.024855,0.026200,44.59,47.773758,False,False,False,VITE,USDT
4,SAND/USD,False,True,2023-03-25,0.631600,0.638550,0.736500,52.88,47.507523,False,False,False,SAND,USD
5,LAZIO/USD4,False,True,2023-03-25,3.454500,3.477375,3.665550,59.20,47.496346,False,False,False,LAZIO,USD4
6,1INCH/USD,False,True,2023-03-25,0.506000,0.520250,0.554500,45.14,47.187741,False,False,False,1INCH,USD
7,MANA/USD,False,True,2023-03-25,0.587400,0.597925,0.663150,55.70,47.088597,False,False,False,MANA,USD
8,GAL/USD,False,True,2023-03-25,1.717000,3.272750,4.676000,42.76,47.071714,False,False,False,GAL,USD
9,JASMY/USDT,False,True,2023-03-25,0.005021,0.003933,0.014750,50.29,46.706027,False,False,False,JASMY,USDT


In [12]:
# plots for bottom10
[plot(symbol) for i,symbol in bottom10.Symbol.items()]

[None, None, None, None, None, None, None, None, None, None]

# Save in CSV Format

In [13]:
# save to csv
datestr = datetime.now().strftime("%Y%m%d")
results.to_csv('DailySignals_{}.csv'.format(datestr),index=False)
print("CSV now in Outputs </kaggle/working/...>")

CSV now in Outputs </kaggle/working/...>

# Supplementary

In [14]:
# Prints the indicators and utility functions
results.ta.indicators()

# Returns a list of indicators and utility functions
ind_list = results.ta.indicators(as_list=True)

# Returns a list of the indicators and utility functions that are not in the excluded list
smaller_list = results.ta.indicators(exclude=["cg", "pgo", "ui"], as_list=True)

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

# Trade Automation Checklist

1. Retrieve currency balances: USD, USDT, BUSD, BTC, ETH, ...
2. Use top10 & bottom10 as list of tradeable symbols.
3. Trade cash balance divided by number of symbols in long positions. trade_amount = (balance / len(top10.loc(top10.Long == True))
4. React to bottom10 short positions first. Sell trade_amount of each symbol in bottom10.loc(bottom10.Short == True)
5. For top10 long positions. Trade with Deno > trade_amount. 

In [15]:
# en fin